In [1]:
# Import dependencies
import pandas as pd
import requests
from database_connection import ENGINE
from config import api_key

In [2]:
# import yelpAPI
from yelpapi import YelpAPI
import argparse
from pprint import pprint

In [3]:
# grab the top 1000 US cities from the database
top_us_cities_query = '''
SELECT *
FROM top_us_cities
'''
cities = pd.read_sql(
    top_us_cities_query,
    con=ENGINE
)

In [4]:
# create a sample DataFrame of 25 cities for initial testing purposes
cities = cities.sample(25)

In [5]:
# create list of city, state combinations
cities_list = cities['city'] + ', ' + cities['abbr']
cities_list

564             Coachella, CA
700                Newark, OH
760            Des Moines, IA
69                  Chino, CA
903               Turlock, CA
837    Palm Beach Gardens, FL
205              Kentwood, MI
320               Madison, AL
139          Calumet City, IL
841            Alpharetta, GA
424             Royal Oak, MI
335            Santa Rosa, CA
856            St. George, UT
124                  Brea, CA
152                Conroe, TX
99              Flagstaff, AZ
536             Inglewood, CA
314               Hayward, CA
638              Hesperia, CA
983             Davenport, IA
306            Sugar Land, TX
736                Lorain, OH
96                 Burien, WA
288                 Parma, OH
377               Findlay, OH
dtype: object

In [6]:
# Create empty list to hold all breweries
brewery_results = []

In [7]:
# Function to pull breweries for each location):
def get_breweries(location):
    with YelpAPI(api_key) as yelp_api:
        # city_data is the json file with all businesses returned from search
        city_data = yelp_api.search_query(term='Breweries', location = location)
        # For each brewery in the search, return business name, coordinates, city, and buisness type
        for i in range(len(city_data['businesses'])):
            yelp_id = city_data['businesses'][i]['id']
            name = city_data['businesses'][i]['name']
            lat = city_data['businesses'][i]['coordinates']['latitude']
            long = city_data['businesses'][i]['coordinates']['longitude']
            city = city_data['businesses'][i]['location']['city']
            business_type = city_data['businesses'][i]['categories'][0]['alias']
            is_closed = city_data['businesses'][i]['is_closed']
            rating = city_data['businesses'][i]['rating']
            zip_code = city_data['businesses'][i]['location']['zip_code']
            state = city_data['businesses'][i]['location']['state']

            brewery_results.append(
                {
                    'yelp_id':yelp_id,
                    'name':name, 
                    'lat':lat, 
                    'lng': long, 
                    'city':city, 
                    'business_type':business_type, 
                    'is_closed':is_closed, 
                    'rating':rating, 
                    'zip_code':zip_code, 
                    'state':state
                }
            )

In [8]:
# Apply get_breweries function to each location in cities_list
for city in cities_list:
    try:
        get_breweries(city)
    except Exception as e:
        print(e)
        print(f'An exception occurred for {city}')

In [9]:
breweries_df = pd.DataFrame(brewery_results)
breweries_df.head()

,yelp_id,name,lat,lng,city,business_type,is_closed,rating,zip_code,state
0,Be9YQy-QQzw8kjeMyFRMBA,Coachella Craft Beer Barn,33.678449,-116.237155,Indio,beer_and_wine,False,4.0,92201,CA
1,4eY-hs8mdt68EhtlFAv8nQ,Taproom 29,33.712645,-116.184130,Coachella,newamerican,False,3.0,92236,CA
2,0YScexYgQzkOPw0vNuia0A,Big Rock Golf and Pub,33.715190,-116.269910,Indio,tradamerican,False,3.5,92201,CA
3,W8m3jQ06xHwTLuMZfZ8UjQ,Parkway Market & Spirits,33.729940,-116.198760,Indio,beer_and_wine,False,4.5,92203,CA
4,_o5YWmeMmgrzd_JvVgmA5Q,Burgers & Beer,33.707135,-116.272040,La Quinta,tradamerican,False,3.5,92253,CA


In [10]:
# Length of breweries_df
len(breweries_df)

421

In [11]:
# Create empty List ot hold all Outdoor Gear Stores
outdoor_gear_results = []

In [12]:
# Function to pull stores from yelp with location as the variable
def get_outdoor_stores(location):
    with YelpAPI(api_key) as yelp_api:
        # city_data is the json file with all businesses returned from search
        city_data = yelp_api.search_query(term='Outdoor Gear', location = location)
        # For each brewery in the search, return business name, coordinates, city, and buisness type
        for i in range(len(city_data['businesses'])):
            yelp_id = city_data['businesses'][i]['id']
            name = city_data['businesses'][i]['name']
            lat = city_data['businesses'][i]['coordinates']['latitude']
            long = city_data['businesses'][i]['coordinates']['longitude']
            city = city_data['businesses'][i]['location']['city']
            business_type = city_data['businesses'][i]['categories'][0]['alias']
            is_closed = city_data['businesses'][i]['is_closed']
            rating = city_data['businesses'][i]['rating']
            zip_code = city_data['businesses'][i]['location']['zip_code']
            state = city_data['businesses'][i]['location']['state']
            
            outdoor_gear_results.append(
                {
                    'yelp_id':yelp_id,
                    'name':name, 
                    'lat':lat, 
                    'lng': long, 
                    'city':city, 
                    'business_type':business_type, 
                    'is_closed':is_closed, 
                    'rating':rating, 
                    'zip_code':zip_code, 
                    'state':state
                }
            )

In [13]:
# Apply get_outdoor_stores function to each location in the cities_list
for city in cities_list:
    try:
        get_outdoor_stores(city)
    except Exception as e:
        print(e)
        print(f'An exception occurred for {city}')

In [14]:
# Convert to pandas DataFrame
outdoor_df = pd.DataFrame(outdoor_gear_results)
outdoor_df.head()

,yelp_id,name,lat,lng,city,business_type,is_closed,rating,zip_code,state
0,8y2LjgZAkUbCZYQTdFXzWg,Yellow Mart Stores,33.721600,-116.219711,Indio,deptstores,False,3.0,92201,CA
1,U-t1cpKQkEHBaX4dxYMOig,Turner's Outdoorsman-Palm Desert,33.767465,-116.312332,Palm Desert,guns_and_ammo,False,4.0,92211,CA
2,G9nZt-5uNXJbwZ0pCU5Wdw,DICK'S Sporting Goods,33.724108,-116.395622,Palm Desert,sportswear,False,2.5,92260,CA
3,AyyrWPxHqvChApd6PNmKuw,Tri-A-Bike,33.722567,-116.382986,Palm Desert,bikes,False,4.0,92260,CA
4,J_fa6g-eQhj_wmn3DJHeog,The Climbing Life Guides,34.111679,-116.330658,Joshua Tree,climbing,False,5.0,92252,CA


In [15]:
# Length of outdoor_df
len(outdoor_df)

369

In [16]:
# fill `breweries` and `outdoor` tables with Yelp search results 
breweries_df.to_sql(
    'breweries', 
    con=ENGINE, 
    if_exists='replace',
    index=False
    )

In [17]:
outdoor_df.to_sql(
    'outdoor', 
    con=ENGINE, 
    if_exists='replace',
    index=False
    )